In [1]:
import pandas as pd

In [2]:



def get_drive_time(x):
    col_map = {
        'B08303_002E': 5,
        'B08303_003E': 7,	
        'B08303_004E': 12,  
        'B08303_005E': 17,
        'B08303_006E': 22,
        'B08303_007E': 27,
        'B08303_008E': 32,
        'B08303_009E': 37,
        'B08303_010E': 42,
        'B08303_011E': 52,
        'B08303_012E': 75,
        'B08303_013E': 90
    }
    cumsum = x['B08303_002E']
    field = 2
    while cumsum < x['B08303_001E'] * .5 and field < 13:
        cumsum += x['B08303_0{}E'.format(str(field).zfill(2))]
        field += 1
    return col_map['B08303_0{}E'.format(str(field).zfill(2))]

def get_education(x, maxf):
    cols = [
        ( 'B15002_003E','B15002_020E','No schooling completed')
        ,('B15002_004E','B15002_021E','Nursery to 4th grade')
        ,('B15002_005E','B15002_022E','5th and 6th grade')
        ,('B15002_006E','B15002_023E','7th and 8th grade')
        ,('B15002_007E','B15002_024E','9th grade')
        ,('B15002_008E','B15002_025E','10th grade')
        ,('B15002_009E','B15002_026E','11th grade')
        ,('B15002_010E','B15002_027E','12th grade, no diploma')
        ,('B15002_011E','B15002_028E','High school graduate (includes equivalency)')
        ,('B15002_012E','B15002_029E','Some college, less than 1 year')
        ,('B15002_013E','B15002_030E','Some college, 1 or more years, no degree')
        ,('B15002_014E','B15002_031E',"Associate's degree")
        ,('B15002_015E','B15002_032E',"Bachelor's degree")
        ,('B15002_016E','B15002_033E',"Master's degree")
        ,('B15002_017E','B15002_034E','Professional school degree')
        ,('B15002_018E','B15002_035E','Doctorate degree')
    ][::-1]

    total = x['B15002_001E']

    cumsum = 0
    for col in cols:
        cumsum += (x[col[0]] + x[col[1]])
        if col[2] == maxf:
            break
    return cumsum / total if total else None
def get_unemployment(x):
    unemployment_fields = [
        'B12006_006E',
        'B12006_011E',
        'B12006_017E',
        'B12006_022E',
        'B12006_028E',
        'B12006_033E',
        'B12006_039E',
        'B12006_044E',
        'B12006_050E',
        'B12006_055E'
    ]

    labor_force_fields = [
        'B12006_004E',
        'B12006_009E',
        'B12006_015E',
        'B12006_020E',
        'B12006_026E',
        'B12006_031E',
        'B12006_037E',
        'B12006_042E',
        'B12006_048E',
        'B12006_053E'
    ]
    denom = sum([x[f] for f in labor_force_fields])
    uer = sum([x[f] for f in unemployment_fields]) /  denom if denom else None
    return uer


In [3]:



field_map = {
    'B24031_003E': 'med_inc_agriculture_forestry_fishing_hunting', 
    'B24031_004E': 'med_inc_mining_quarrying_oil_gas_extraction', 
    'B24031_005E': 'med_inc_construction', 
    'B24031_006E': 'med_inc_manufacturing', 
    'B24031_007E': 'med_inc_wholesale_trade', 
    'B24031_008E': 'med_inc_retail_trade', 
    'B24031_010E': 'med_inc_transportation_warehousing', 
    'B24031_011E': 'med_inc_utilities', 
    'B24031_012E': 'med_inc_information', 
    'B24031_014E': 'med_inc_finance_insurance', 
    'B24031_015E': 'med_inc_real_estate_rental_leasing', 
    'B24031_017E': 'med_inc_professional_scientific_technical_services', 
    'B24031_018E': 'med_inc_management_of_companies_enterprises', 
    'B24031_019E': 'med_inc_administrative_support_waste_management_services', 
    'B24031_021E': 'med_inc_educational_services', 
    'B24031_022E': 'med_inc_health_care_social_assistance', 
    'B24031_024E': 'med_inc_arts_entertainment_recreation', 
    'B24031_025E': 'med_inc_accommodation_food_services', 
    'B24031_026E': 'med_inc_other_services_except_public_administration', 
    'B24031_027E': 'med_inc_public_administration',
    'B01001_001E': 'total_population',
    'B01002_001E': 'median_age',
    'B19013_001E': 'median_household_income',
    'B19301_001E': 'per_capita_income',
    'B23020_001E': 'mean_hours_worked',
    'B25058_001E': 'Median_contract_rent',
    'B25105_001E': 'Median_monthly_housing_costs',
    'B25071_001E': 'med_gross_rent_pct_hshld_income',
}

#  'B01001A_001E': 'population_white',

functions = {
        'median_commute': get_drive_time,
        'high_school': lambda x: get_education(x, 'High school graduate (includes equivalency)'),
        'associates': lambda x: get_education(x, "Associate's degree"),
        'bachelors': lambda x: get_education(x, "Bachelor's degree"),
        'masters': lambda x: get_education(x, "Master's degree"),
        'doctorate': lambda x: get_education(x, "Doctorate degree"),
        'unemployment_rate': get_unemployment,
        'pct_minority': lambda x: 1 - (x['B01001A_001E'] / x['B01001_001E']) if x['B01001_001E'] else None
    }

geos = [
    ('state','state',40),
    ('county','county',50),
    ('place','place',160),
    ('metro', 'metropolitan/micropolitan statistical area',310)
]

geo_codes = {
    'state':40,
    'county':50,
    'place':160,
    'metro': 310
}

null_cols = [
    'crime_per_100k',
    'crime_index',
    'annual_cost',
    'cost_index'
]

In [4]:
import re
geos = [
    'state',
    'county',
    'place'
]
geos = ['metro']

years = list(range(2010,2021))
# years = [2021]

for year in years:
    for geo in geos:

        f = f'data/{geo}_{year}_census_api.csv'
        outf = f'data/processed/{geo}_{year}_census_api_p.csv'

        df = pd.read_csv(f)

        for name, func in functions.items():
            df[name] = df.apply(func, axis=1)

        df = df.rename(field_map, axis=1)

        if 'SUMLEVEL' not in df.columns:
            df['SUMLEVEL'] = geo_codes[geo]

        for col in null_cols:
            if col not in df.columns:
                df[col] = None

        drop_cols = filter(lambda x:bool(re.search("^B.*_.*E$", x)), df.columns)
        df = df.drop(drop_cols,axis=1)
        
        df.to_csv(outf, index=False)

In [55]:
df = pd.read_csv('/home/shumway743/cis5500_group/census/data/processed/county_2021_census_api_p.csv')
idx = pd.read_csv('/home/shumway743/cis5500_group/census/2021.csv')
df = df.set_index('GEO_ID')
df = df.drop(null_cols, axis=1)
idx = idx[['GEO_ID','SUMLEVEL'] + null_cols].set_index('GEO_ID')

idx[idx.SUMLEVEL == 50].count()
# jdf = df.join(idx, how='left')

# jdf.head()
#df.sort_values('col',ascending=False).drop_duplicates([cols])

/tmp/ipykernel_7061/1352093562.py:2: DtypeWarning: Columns (4,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  idx = pd.read_csv('/home/shumway743/cis5500_group/census/2021.csv')


SUMLEVEL          14614
crime_per_100k    14525
crime_index       14525
annual_cost       14531
cost_index        14531
dtype: int64

In [52]:
idx.count()

crime_per_100k    22823
crime_index       22823
annual_cost       15111
cost_index        15111
dtype: int64

In [46]:
{
    20:'Region',
    30:'Division',
    40:'State',
    50:'State-County',
    60:'State-County-County Subdivision',
    67:'State-County-County Subdivision-Subminor Civil Division',
    140:'State-County-Census Tract',
    150:'State-County-Census Tract-Block Group',
    160:'State-Place',
    170:'State-Consolidated City',
    230:'State-Alaska Native Regional Corporation',
    250:'American Indian Area/Alaska Native Area/Hawaiian Home Land',
    251:'American Indian Area-Tribal Subdivision/Remainder',
    252:'American Indian Area/Alaska Native Area (Reservation or Statistical Entity Only)',
    254:'American Indian Area (Off-Reservation Trust Land Only)/Hawaiian Home Land',
    256:'American Indian Area-Tribal Census Tract',
    258:'American Indian Area-Tribal Census Tract-Tribal Block Group',
    310:'Metropolitan Statistical Area/Micropolitan Statistical Area',
    314:'Metropolitan Statistical Area-Metropolitan Division',
    330:'Combined Statistical Area',
    332:'Combined Statistical Area-Metropolitan Statistical Area/Micropolitan Statistical Area',
    335:'Combined New England City and Town Area',
    337:'Combined New England City and Town Area-New England City and Town Area',
    350:'New England City and Town Area',
    352:'New England City and Town Area-State-Principal City',
    355:'New England City and Town Area (NECTA)-NECTA Division',
    361:'State-New England City and Town Area-Principal City',
    500:'State-Congressional District (111th)',
    610:'State-State Legislative District (Upper Chamber)',
    620:'State-State Legislative District (Lower Chamber)',
    700:'State-County-Voting District/Remainder',
    860:'5-Digit ZIP code Tabulation Area',
    950:'State-School District (Elementary)/Remainder',
    960:'State-School District (Secondary)/Remainder',
    970:'State-School District (Unified)/Remainder'
}

keep = [40,50,140,160,310]

In [32]:
import json

with open('/home/shumway743/cis5500_group/census/variables.json','r') as f:
    vals = json.load(f)

term = 'fuel'
# print([x for x in vals['variables'].values()])
filt = [x for x in vals['variables'].values() if term in x['label'].lower() or ('concept' in x and 'cost' in x['concept'].lower())]

print(json.dumps(filt, indent=4))

[
    {
        "label": "2017-2021 Estimates!!HOUSE HEATING FUEL!!Occupied housing units!!No fuel used",
        "concept": "COMPARATIVE HOUSING CHARACTERISTICS",
        "predicateType": "float",
        "group": "CP04",
        "limit": 0,
        "attributes": "CP04_2021_071EA,CP04_2021to2016_071SS"
    },
    {
        "label": "2017-2021 Estimates!!HOUSE HEATING FUEL!!Occupied housing units!!Other fuel",
        "concept": "COMPARATIVE HOUSING CHARACTERISTICS",
        "predicateType": "float",
        "group": "CP04",
        "limit": 0,
        "attributes": "CP04_2021_070EA,CP04_2021to2016_070SS"
    },
    {
        "label": "2017-2021 Estimates!!HOUSE HEATING FUEL!!Occupied housing units!!Solar energy",
        "concept": "COMPARATIVE HOUSING CHARACTERISTICS",
        "predicateType": "float",
        "group": "CP04",
        "limit": 0,
        "attributes": "CP04_2021_069EA,CP04_2021to2016_069SS"
    },
    {
        "label": "2017-2021 Estimates!!HOUSE HEATING FUEL!!Occup